In [ ]:
import torch
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import skimage
from importlib import reload
import os
import folded_dataset
reload(folded_dataset)
import utils
reload(utils)
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score, classification_report

import evaluation

In [ ]:
root_dir = '/scr/zchen/datasets/morphem_70k_2.0'
dataset = 'CP'
model_choice = 'knn' # 'knn' or 'sgd'

leave_out = 'Task_four' # Leave-one-out task, set to None for Allen, 'Task_three' for HPA, 'Task_four' for CP
leaveout_label = 'Plate' # Leave-one-out column name, 'cell_type' for HPA, 'Plate' for CP

dest_dir = f'results' # directory to save results
save_csv = True # Set to True when running the last dataset to save result csv

In [ ]:
features_path = f'{root_dir}/features/{dataset}/pretrained_resnet18_features.npy'
df_path = f'{root_dir}/{dataset}/enriched_meta.csv'

results = evaluation.evaluate(features_path, df_path, leave_out, leaveout_label, model_choice)

print('Results:')
for task_ind, task in enumerate(results["tasks"]):
    print(f'Results for {dataset} {task} with {model_choice} :')
    print(results["reports_str"][task_ind])

In [ ]:
full_reports_dict = {}
full_reports_dict['target_encoding'] = results["encoded_target"]
for task_ind, task in enumerate(results["tasks"]):
    full_reports_dict[task] = results["reports_dict"][task_ind]


In [ ]:
if not os.path.exists(dest_dir+ '/'):
    os.makedirs(dest_dir+ '/')
    
dict_path = f'{dest_dir}/{dataset}_{model_choice}_full_results.json'
with open(dict_path, 'w') as f:
    json.dump(full_reports_dict, f)

In [ ]:
if 'results_df' in locals():
    results_temp = pd.DataFrame({'source': [dataset for i in range(len(results["tasks"]))],\
                        'task': results["tasks"],'model': [model_choice for i in range(len(results["tasks"]))],\
                        'accuracy': results["accuracies"],'f1_score_macro': results["f1scores_macro"]})
    results_df = pd.concat([results_df, results_temp]).reset_index(drop=True)

else: 
    results_df = pd.DataFrame({'source': [dataset for i in range(len(results["tasks"]))],\
                        'task': results["tasks"],'model': [model_choice for i in range(len(results["tasks"]))],\
                        'accuracy': results["accuracies"],'f1_score_macro': results["f1scores_macro"]})

In [ ]:
if save_csv:
    results_df.to_csv(f'{dest_dir}/resnet18_knn_sgd.csv', index=False)